# TP 1
## Groupe des amateurices de vin

### Reconnaissance de vin.

Dans ce projet, nous allons faire de la reconnaissance de vins.

Source des données : https://huggingface.co/datasets/katossky/wine-recognition

In [4]:
import torch
from torch import nn
from torchvision.io import read_image
import numpy as np
import glob
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

### Étape 1 : Charger les données

En PyTorch, les données doivent être transmise au réseau de neurones à l'aide d'un loader. La première étape est de créer une classe de type `Dataset` que la fonction `DataLoader` prend en argument. La classe doit au moins posséder les trois routine `__init__`, `__len__` et `__getitem__`.

In [8]:
import pandas as pd

dataset = pd.read_csv('wine-recognition/wine.csv')

print(dataset)

     label  alcohol  malic acid   ash  alcalinity of ash  magnesium  \
0        1    14.23        1.71  2.43               15.6        127   
1        1    13.20        1.78  2.14               11.2        100   
2        1    13.16        2.36  2.67               18.6        101   
3        1    14.37        1.95  2.50               16.8        113   
4        1    13.24        2.59  2.87               21.0        118   
..     ...      ...         ...   ...                ...        ...   
173      3    13.71        5.65  2.45               20.5         95   
174      3    13.40        3.91  2.48               23.0        102   
175      3    13.27        4.28  2.26               20.0        120   
176      3    13.17        2.59  2.37               20.0        120   
177      3    14.13        4.10  2.74               24.5         96   

     total phenols  flavanoids  nonflavanoid phenols  proanthocyanins  \
0             2.80        3.06                  0.28             2.29   
1